# Using continous / categorical / transition matrices to get recommendations / Selective Box Cox

In [2]:
import pandas as pd
from sklearn.cluster import KMeans
import seaborn as sns
from matplotlib import pyplot as plt 
import math
from ipynb.fs.full.Utils import createKDeplot, loadDataFrames, dropFeaturesWithNoVariance
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy import stats
from ipynb.fs.full.Pipeline import removeConstantFeatures, applyBoxCoxAllFeatures, centerAndScale, applyEuclideanDistance, getIndexAndVector, applyBoxCoxFeatures

In [3]:
transitionMatrices = pd.read_csv('./data/07-transitionsClustered.csv')
spotifyFeatures = pd.read_csv('./data/continousFeatures.csv') # should be gone
spotifyFeatures = spotifyFeatures.iloc[:, 1:]

In [6]:
transitionMatrices

,id,0,1,2,4,5,6,7,8,9,...,1278,1281,1286,1287,1288,1290,1292,1294,1295,category
0,23,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1
1,105,0.043232,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,2
2,118,0.000000,0.0,0.0,0.0,0.0,0.003229,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1
3,120,0.021256,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1
4,292,0.011848,0.0,0.0,0.0,0.0,0.000000,0.000923,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3506,5000025,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,2
3507,5000026,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000308,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1
3508,5000027,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,2
3509,5000028,0.010382,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,2


In [4]:

data = pd.read_csv('./data/05-allDataContinousCategoricalTransition.csv')

In [5]:
data

,Unnamed: 0,index,id,danceability,energy,speechiness,acousticness,liveness,valence,tempo,...,mood_5,mood_6,mood_7,mood_8,mood_9,mood_10,mood_11,instrumentalness_cat_0,instrumentalness_cat_1,instrumentalness_cat_2
0,0,0,23,0.494,0.756,0.0432,0.001400,0.1640,0.589,133.337,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,1,105,0.582,0.744,0.0336,0.000458,0.0405,0.674,133.344,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,2,2,118,0.614,0.755,0.0276,0.016200,0.0831,0.645,92.622,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,3,3,120,0.319,0.807,0.0388,0.001040,0.1310,0.601,118.097,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,4,4,292,0.496,0.527,0.0339,0.195000,0.1380,0.283,155.932,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,3195,3488,5000017,0.453,0.295,0.0291,0.518000,0.2930,0.140,146.022,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3196,3196,3489,5000018,0.525,0.292,0.0260,0.910000,0.0795,0.133,75.009,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3197,3197,3490,5000019,0.549,0.283,0.0284,0.661000,0.3570,0.163,76.992,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3198,3198,3491,5000020,0.493,0.270,0.0278,0.244000,0.1230,0.176,73.001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [7]:
df = pd.merge(spotifyFeatures, transitionMatrices, on='id') # Not needed should be gone 

In [4]:
originalSongId = 180849 # The Scientist Original
categoryOriginalSong = df[df.id == originalSongId].category.values[0]
df = df[df.category == categoryOriginalSong]

In [5]:
df

,id,danceability,energy,speechiness,acousticness,liveness,valence,tempo,0,1,...,1278,1281,1286,1287,1288,1290,1292,1294,1295,category
1,105,0.582,0.744,0.0336,0.000458,0.0405,0.674,133.344,0.043232,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
12,816,0.536,0.928,0.0589,0.015400,0.2000,0.389,136.258,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
13,830,0.526,0.665,0.0243,0.407000,0.1690,0.285,158.493,0.043169,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
17,1395,0.561,0.593,0.0264,0.003270,0.0967,0.574,80.282,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
18,1433,0.481,0.638,0.0276,0.153000,0.1540,0.497,79.064,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3194,5000013,0.334,0.252,0.0287,0.722000,0.0907,0.214,153.879,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3195,5000017,0.453,0.295,0.0291,0.518000,0.2930,0.140,146.022,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3196,5000018,0.525,0.292,0.0260,0.910000,0.0795,0.133,75.009,0.022645,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3197,5000019,0.549,0.283,0.0284,0.661000,0.3570,0.163,76.992,0.037170,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2


In [6]:
df[df.id > 5000000]

,id,danceability,energy,speechiness,acousticness,liveness,valence,tempo,0,1,...,1278,1281,1286,1287,1288,1290,1292,1294,1295,category
3189,5000001,0.255,0.343,0.0310,0.927,0.1640,0.1160,140.527,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3190,5000007,0.273,0.205,0.0316,0.941,0.1710,0.0796,148.225,0.036094,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3191,5000008,0.506,0.481,0.0264,0.482,0.2000,0.2240,148.626,0.010382,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3192,5000011,0.503,0.123,0.0333,0.879,0.1140,0.2800,145.843,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3193,5000012,0.306,0.229,0.0331,0.807,0.1120,0.1560,85.492,0.029018,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3194,5000013,0.334,0.252,0.0287,0.722,0.0907,0.2140,153.879,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3195,5000017,0.453,0.295,0.0291,0.518,0.2930,0.1400,146.022,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3196,5000018,0.525,0.292,0.0260,0.910,0.0795,0.1330,75.009,0.022645,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3197,5000019,0.549,0.283,0.0284,0.661,0.3570,0.1630,76.992,0.037170,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3198,5000020,0.493,0.270,0.0278,0.244,0.1230,0.1760,73.001,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2


In [7]:
df = df.drop(columns=['category'])

In [8]:
df.reset_index(drop=True, inplace=True)

In [9]:
df

,id,danceability,energy,speechiness,acousticness,liveness,valence,tempo,0,1,...,1276,1278,1281,1286,1287,1288,1290,1292,1294,1295
0,105,0.582,0.744,0.0336,0.000458,0.0405,0.674,133.344,0.043232,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,816,0.536,0.928,0.0589,0.015400,0.2000,0.389,136.258,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,830,0.526,0.665,0.0243,0.407000,0.1690,0.285,158.493,0.043169,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1395,0.561,0.593,0.0264,0.003270,0.0967,0.574,80.282,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1433,0.481,0.638,0.0276,0.153000,0.1540,0.497,79.064,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899,5000013,0.334,0.252,0.0287,0.722000,0.0907,0.214,153.879,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
900,5000017,0.453,0.295,0.0291,0.518000,0.2930,0.140,146.022,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
901,5000018,0.525,0.292,0.0260,0.910000,0.0795,0.133,75.009,0.022645,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902,5000019,0.549,0.283,0.0284,0.661000,0.3570,0.163,76.992,0.037170,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
## Removing unamed and other unnecessary columns
df = df.sort_values('id')
df.id = df.id.astype('int64')
# Saving idChords
ids = df.id
df = df.set_index('id')

In [11]:
ids

0          105
1          816
2          830
3         1395
4         1433
        ...   
899    5000013
900    5000017
901    5000018
902    5000019
903    5000020
Name: id, Length: 904, dtype: int64

## Pipeline

In [12]:
df = removeConstantFeatures(df)
df= applyBoxCoxFeatures(df, ['speechiness', 'acousticness', 'liveness', 'valence'])
df = centerAndScale(df)

In [13]:
df

,danceability,energy,speechiness,acousticness,liveness,valence,tempo,0,1,2,...,1256,1260,1266,1271,1274,1275,1281,1287,1292,1295
0,0.461078,0.752810,-0.185045,-1.787120,-2.887895,1.195615,0.436969,2.087169,-0.057703,-0.033278,...,-0.047088,-0.057703,-0.033278,-0.033278,-0.033278,-0.033278,-0.074577,-0.047088,-0.057703,-0.033278
1,0.116399,1.590304,1.530708,-0.999153,0.700590,0.117197,0.537443,-0.686703,-0.057703,-0.033278,...,-0.047088,-0.057703,-0.033278,-0.033278,-0.033278,-0.033278,-0.074577,-0.047088,-0.057703,-0.033278
2,0.041469,0.393235,-2.026056,0.709880,0.436133,-0.399141,1.304100,2.083140,-0.057703,-0.033278,...,-0.047088,-0.057703,-0.033278,-0.033278,-0.033278,-0.033278,-0.074577,-0.047088,-0.057703,-0.033278
3,0.303724,0.065520,-1.470517,-1.432035,-0.606252,0.856543,-1.392595,-0.686703,-0.057703,-0.033278,...,-0.047088,-0.057703,-0.033278,-0.033278,-0.033278,-0.033278,-0.074577,-0.047088,-0.057703,-0.033278
4,-0.295717,0.270341,-1.198820,0.046334,0.280980,0.569798,-1.434591,-0.686703,-0.057703,-0.033278,...,-0.047088,-0.057703,-0.033278,-0.033278,-0.033278,-0.033278,-0.074577,-0.047088,-0.057703,-0.033278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899,-1.397191,-1.486574,-0.973909,1.181148,-0.744273,-0.822877,1.145010,-0.686703,-0.057703,-0.033278,...,-0.047088,-0.057703,-0.033278,-0.033278,-0.033278,-0.033278,-0.074577,-0.047088,-0.057703,-0.033278
900,-0.505522,-1.290855,-0.897243,0.899950,1.228109,-1.370370,0.874103,-0.686703,-0.057703,-0.033278,...,-0.047088,-0.057703,-0.033278,-0.033278,-0.033278,-0.033278,-0.074577,-0.047088,-0.057703,-0.033278
901,0.033976,-1.304510,-1.567922,1.391409,-1.041659,-1.430684,-1.574407,0.766256,-0.057703,-0.033278,...,-0.047088,-0.057703,-0.033278,-0.033278,-0.033278,-0.033278,-0.074577,-0.047088,-0.057703,-0.033278
902,0.213808,-1.345474,-1.033142,1.104101,1.466098,-1.184334,-1.506033,1.698206,-0.057703,-0.033278,...,-0.047088,-0.057703,-0.033278,-0.033278,-0.033278,-0.033278,-0.074577,-0.047088,-0.057703,-0.033278


### Original Song & Cover Song

In [14]:
originalSongId = 180849 # The Scientist Original
originalSongIndex,originalSongVector = getIndexAndVector(originalSongId, ids, df)

In [16]:
coverSongId = 5000013 # The Scientist Original
coverSongIndex,coverSongVector = getIndexAndVector(coverSongId, ids, df)

### Get Rankings

In [18]:
ranksDF  = applyEuclideanDistance(df, ids, originalSongVector)

In [19]:
ranksDF.head(10)

,id,rank
0,180849,0.000000
1,610562,33.443224
2,732460,34.574651
3,826695,34.709058
4,2689908,34.962459
5,937028,34.996228
6,5000019,34.998764
7,758267,35.009505
8,1065079,35.012978
9,1026669,35.030269


In [20]:
ranksDF[ranksDF['id']==coverSongId]

,id,rank
85,5000013,35.270836


### Cover songs

In [21]:
ranksDF[ranksDF['id'] > 5000000]

,id,rank
6,5000019,34.998764
12,5000017,35.048895
85,5000013,35.270836
302,5000012,35.545787
644,5000011,36.824128
679,5000018,37.432094
789,5000001,42.402148
842,5000020,49.169136
854,5000008,52.588457
873,5000007,59.219768
